In [12]:
! py -m pip install mlxtend --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.4.1 requires packaging<22.0,>=0.21, but you have packaging 23.2 which is incompatible.



  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 1.4/1.4 MB 23.2 MB/s eta 0:00:00
   ---------------------------------------- 11.1/11.1 MB 18.8 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.1.7
    Uninstalling mlxtend-0.1.7:
      Successfully uninstalled mlxtend-0.1.7


In [ ]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta

# Setup
fake = Faker()
np.random.seed(42)
random.seed(42)

# -----------------------
# 1. Generate Customers
# -----------------------
num_customers = 100
customer_data = []

for i in range(1, num_customers + 1):
    is_male = random.choice([True, False])
    first_name = fake.first_name_male() if is_male else fake.first_name_female()
    last_name = fake.last_name()
    gender = 'M' if is_male else 'F'
    dob = fake.date_of_birth(minimum_age=18, maximum_age=75)
    age = (datetime.now().date() - dob).days // 365
    income = random.randint(20000, 150000)
    credit_score = int(np.clip(np.random.normal(loc=600 + (income / 1000), scale=50), 300, 850))

    customer_data.append({
        "Customer_ID": f"CUST{i:04d}",
        "First_Name": first_name,
        "Last_Name": last_name,
        "Gender": gender,
        "Date_of_Birth": dob,
        "Age": age,
        "Email": fake.email(),
        "Phone": fake.phone_number(),
        "Address": fake.address().replace("\n", ", "),
        "City": fake.city(),
        "State": fake.state(),
        "Postal_Code": fake.postcode(),
        "Country": "USA",
        "Annual_Income": income,
        "Credit_Score": credit_score
    })

df_customers = pd.DataFrame(customer_data)
df_customers.to_csv("customers.csv", index=False)

# ----------------------------
# 2. Generate Transactions
# ----------------------------
transaction_data = []
categories = ['Grocery', 'Fuel', 'Dining', 'Travel', 'Medical', 'Baby', 'Education', 'Entertainment', 'Home', 'Salary']
merchants = {
    'Grocery': ['Walmart', 'Whole Foods', 'Kroger'],
    'Fuel': ['Shell', 'Exxon', 'Chevron'],
    'Dining': ['McDonald\'s', 'Starbucks', 'Chipotle'],
    'Travel': ['Delta', 'Airbnb', 'Uber'],
    'Medical': ['CVS', 'Walgreens', 'Urgent Care'],
    'Baby': ['BabyGap', 'Target', 'Pampers Store'],
    'Education': ['Coursera', 'Udemy', 'University Bookstore'],
    'Entertainment': ['Netflix', 'AMC Theatres', 'Spotify'],
    'Home': ['Home Depot', 'Lowe\'s', 'IKEA'],
    'Salary': ['Company Payroll', 'Direct Deposit']
}

txn_id = 1
for customer in df_customers['Customer_ID']:
    num_txns = random.randint(5, 15)
    for _ in range(num_txns):
        category = random.choice(categories)
        merchant = random.choice(merchants[category])
        amount = round(random.uniform(10, 2000) if category != 'Salary' else random.uniform(3000, 10000), 2)
        date = datetime.now() - timedelta(days=random.randint(0, 28))
        
        transaction_data.append({
            "Transaction_ID": f"TXN{txn_id:06d}",
            "Customer_ID": customer,
            "Timestamp": date.strftime("%Y-%m-%d %H:%M:%S"),
            "Merchant": merchant,
            "Category": category,
            "Amount": amount,
            "Description": f"{category} - {merchant}"
        })
        txn_id += 1

df_transactions = pd.DataFrame(transaction_data)
df_transactions.to_csv("transactions.csv", index=False)

# -------------------------
# 3. Generate Product Table
# -------------------------



TypeError: Random.randint() takes 3 positional arguments but 4 were given

In [3]:
import pandas as pd

# Load CSVs
df_customers = pd.read_csv("customers.csv")
df_transactions = pd.read_csv("transactions.csv")

# Ensure timestamp is in datetime format
df_transactions["Timestamp"] = pd.to_datetime(df_transactions["Timestamp"])

# Filter only last 4 weeks of transactions
latest_date = df_transactions["Timestamp"].max()
cutoff_date = latest_date - pd.Timedelta(days=28)
df_recent = df_transactions[df_transactions["Timestamp"] >= cutoff_date]

# Aggregate features per customer
features = df_recent.groupby("Customer_ID").agg(
    Total_Spend=("Amount", "sum"),
    Num_Transactions=("Transaction_ID", "count"),
    Avg_Txn_Amount=("Amount", "mean"),
    Max_Txn_Amount=("Amount", "max"),
    Has_Salary_Credit=("Category", lambda x: int("Salary" in x.values))
).reset_index()

# Pivot category spend into separate columns
category_spend = df_recent.pivot_table(
    index="Customer_ID",
    columns="Category",
    values="Amount",
    aggfunc="sum",
    fill_value=0
).add_prefix("Spend_").reset_index()

# Merge with customer demographics
df_feature_store = (
    features
    .merge(category_spend, on="Customer_ID", how="left")
    .merge(df_customers[["Customer_ID", "Age", "Annual_Income", "Credit_Score"]], on="Customer_ID", how="left")
)

# Save to CSV
df_feature_store.to_csv("feature_store.csv", index=False)


In [67]:
import pandas as pd

# Full product list with detailed columns, plus structured eligibility fields
products = [
    # Product_ID, Name, Type, Tier,
    # Features_Benefits, Target_Behavior, Special_Offer,
    # min_age, min_income, min_credit_score
    ("P001", "Starter Credit Card", "Credit Card", "Low",
     "1% cashback on all purchases",
     "New credit users, low spenders",
     "USD 25 cashback on first USD 300 spend",
     None,       20000, 600),
    ("P002", "Everyday Saver", "Savings Account", "Low",
     "4% interest, no maintenance fees",
     "Customers starting savings",
     "USD 50 bonus for maintaining USD 1,000 balance for 3 months",
     None,       100,   None),
    ("P003", "Smart Shopper Card", "Credit Card", "Mid",
     "2% cashback on groceries & fuel",
     "High grocery and fuel spenders",
     "USD 50 cashback on first USD 500 spend",
     None,       30000, 650),
    ("P004", "Smart Budget Account", "Savings + Budget", "Mid",
     "5% APY on goal-based savings",
     "Budget-conscious savers",
     "Free premium budgeting tools for 6 months",
     None,       25000, None),
    ("P005", "Travel Rewards Elite", "Credit Card", "High",
     "3x travel points, lounge access",
     "Frequent travelers",
     "USD 200 travel voucher after USD 5,000 spend",
     None,       70000, 700),
    ("P006", "Elite Wealth Plan", "Investment", "High",
     "6–8% ROI, personal wealth advisor",
     "High-income professionals",
     "1% bonus returns for first year",
     30,         100000, None),
    ("P007", "Family Future Plan", "Insurance/Savings", "Mid-High",
     "Child savings + life insurance combo",
     "Families, new parents",
     "USD 100 bonus on first-year premium",
     25,         None,   None),
    ("P008", "Student Flex Account", "Savings", "Student",
     "No fees, education budgeting tools",
     "Students under 25",
     "USD 100 top-up on monthly deposits over USD 200",
     None,       None,   None),  # eligibility purely age<25
    ("P009", "Home Advantage Loan", "Loan", "Mid-High",
     "Low-interest home loans",
     "First-time home buyers",
     "Reduced interest rate for bundled insurance",
     None,       None,   None),
    ("P010", "Retirement Essentials", "Wealth + Health", "Senior",
     "Pension fund + health perks",
     "Customers over 60",
     "USD 500 wellness bonus for first year",
     60,         None,   None),
    ("P011", "FlexFuel Card", "Credit Card", "Low-Mid",
     "3% cashback on fuel",
     "Regular commuters",
     "USD 50 fuel voucher after USD 500 spend",
     None,       None,   None),
    ("P012", "Digital Nomad Saver", "Savings Account", "Mid",
     "1.5% bonus APY for international use",
     "Frequent travelers, remote workers",
     "No international transfer fees for 6 months",
     None,       None,   None),
    ("P013", "HealthSecure Plan", "Health + Savings", "Mid",
     "HSA integration, telehealth access",
     "Health-conscious individuals",
     "USD 200 credit toward medical expenses",
     None,       None,   None),
    ("P014", "GreenLife Investment", "ESG Investment", "Mid-High",
     "5–9% returns in sustainable funds",
     "Eco-conscious investors",
     "USD 100 green bonus + 0.5% bonus returns",
     None,       40000,  None),
    ("P015", "Weekend Explorer Card", "Credit Card", "Mid",
     "2.5% cashback on dining & entertainment",
     "Social and active lifestyle",
     "Free concert ticket after USD 2,000 spend",
     None,       400,    None),
    ("P016", "CashBuilder Certificate", "CD / Fixed Deposit", "Low-Mid",
     "4.75% fixed interest (6 months)",
     "Idle balance savers",
     "USD 25 bonus for auto-renewal",
     None,       5000,   None),
    ("P017", "Lifestyle Bundle Plus", "Bundle (3-in-1)", "High",
     "Credit + Wealth + Travel perks",
     "Affluent, multi-product users",
     "USD 300 statement credit + concierge onboarding",
     None,       100000, None),
    ("P018", "BabyStart Trust Plan", "Child Investment", "Mid",
     "Custodial account for education",
     "Parents with young children",
     "1st-year fees waived + USD 100 education bonus",
     None,       None,   None),
    ("P019", "MoveSmart Relocation Loan", "Loan", "Mid",
     "0% for 6 months, flexible repayment",
     "Customers with large recent spends",
     "USD 150 moving voucher",
     None,       5000,   None),
    ("P020", "SideHustle Account", "Business Checking", "Low-Mid",
     "No fees, invoice management",
     "Freelancers, side businesses",
     "USD 75 bonus for linking payment gateway",
     None,       None,   None),
]

# Create DataFrame with the new structured columns
columns = [
    "Product_ID", "Product_Name", "Product_Type", "Tier",
    "Features_Benefits", "Target_Behavior", "Special_Offer",
    "min_age", "min_income", "min_credit_score"
]

df_products = pd.DataFrame(products, columns=columns)

# Save to CSV (or load into your DB directly)
df_products.to_csv("products.csv", index=False)
print("✅ products.csv created with structured eligibility columns.")


✅ products.csv created with structured eligibility columns.


In [37]:
import pandas as pd

# Load data
df_transactions = pd.read_csv("transactions.csv")
df_customers = pd.read_csv("customers.csv")
df_feature_store = pd.read_csv("feature_store.csv")

# Ensure Timestamp is datetime
df_transactions["Timestamp"] = pd.to_datetime(df_transactions["Timestamp"])

# 1️⃣ Calculate Aggregation_Days per customer
agg_days = df_transactions.groupby("Customer_ID").agg(
    First_Txn_Date=("Timestamp", "min"),
    Last_Txn_Date=("Timestamp", "max")
)
agg_days["Aggregation_Days"] = (agg_days["Last_Txn_Date"] - agg_days["First_Txn_Date"]).dt.days + 1
agg_days = agg_days[["Aggregation_Days"]].reset_index()

# 2️⃣ Calculate Spend Variability (std deviation of Amount)
spend_var = df_transactions.groupby("Customer_ID").agg(
    Spend_Variability=("Amount", "std")
).fillna(0).reset_index()

# 3️⃣ Calculate Salary_to_Spend_Ratio
salary_spend = df_transactions.groupby(["Customer_ID", "Category"]).agg(Total_Category_Spend=("Amount", "sum")).reset_index()
salary = salary_spend[salary_spend["Category"] == "Salary"][["Customer_ID", "Total_Category_Spend"]].rename(columns={"Total_Category_Spend": "Total_Salary"})
total_spend = df_feature_store[["Customer_ID", "Total_Spend"]]

salary_ratio = pd.merge(total_spend, salary, on="Customer_ID", how="left").fillna(0)
salary_ratio["Salary_to_Spend_Ratio"] = salary_ratio["Total_Salary"] / salary_ratio["Total_Spend"]
salary_ratio = salary_ratio[["Customer_ID", "Salary_to_Spend_Ratio"]]

# 4️⃣ Top Spend Category
top_category = df_transactions[df_transactions["Category"] != "Salary"].groupby(["Customer_ID", "Category"]).agg(
    Category_Spend=("Amount", "sum")
).reset_index()

top_spend_cat = top_category.loc[top_category.groupby("Customer_ID")["Category_Spend"].idxmax()]
top_spend_cat = top_spend_cat[["Customer_ID", "Category"]].rename(columns={"Category": "Top_Spend_Category"})

# 5️⃣ Idle Balance Estimate
income = df_customers[["Customer_ID", "Annual_Income"]]
idle_balance = pd.merge(total_spend, income, on="Customer_ID", how="left")
idle_balance["Idle_Balance_Estimate"] = idle_balance["Annual_Income"] - idle_balance["Total_Spend"]
idle_balance = idle_balance[["Customer_ID", "Idle_Balance_Estimate"]]

# Merge all new features
df_enhanced = df_feature_store.merge(agg_days, on="Customer_ID", how="left")
df_enhanced = df_enhanced.merge(spend_var, on="Customer_ID", how="left")
df_enhanced = df_enhanced.merge(salary_ratio, on="Customer_ID", how="left")
df_enhanced = df_enhanced.merge(top_spend_cat, on="Customer_ID", how="left")
df_enhanced = df_enhanced.merge(idle_balance, on="Customer_ID", how="left")

# Fill any remaining NaNs
df_enhanced = df_enhanced.fillna({
    "Spend_Variability": 0,
    "Salary_to_Spend_Ratio": 0,
    "Top_Spend_Category": "Unknown",
    "Aggregation_Days": 28  # Default if only 1 txn
})

# Save enhanced feature store
df_enhanced.to_csv("feature_store_enhanced.csv", index=False)

print("✅ Enhanced feature_store_enhanced.csv created.")


✅ Enhanced feature_store_enhanced.csv created.


In [70]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Load customer data and product catalog
df_customers = pd.read_csv("customers.csv")
df_products = pd.read_csv("products.csv")

# Prepare list to store ownership records
customer_products = []

# Define logic-based exclusions
def get_eligible_products(customer_row):
    age = customer_row['Age']
    eligible_products = df_products.copy()

    # Exclude Student Account for non-students
    if age > 25:
        eligible_products = eligible_products[eligible_products['Product_ID'] != 'P008']
    
    # Exclude Retirement plan for young customers
    if age < 55:
        eligible_products = eligible_products[eligible_products['Product_ID'] != 'P010']
    
    return eligible_products['Product_ID'].tolist()

# Helper to pick a random date in H1 2024
start_date = datetime(2024, 1, 1)
end_date   = datetime(2024, 6, 30)
delta_days = (end_date - start_date).days

def random_h1_2024_date():
    return (start_date + timedelta(days=random.randint(0, delta_days))).date()

# Generate ownership for each customer
for _, customer in df_customers.iterrows():
    cust_id = customer['Customer_ID']
    
    # Get eligible products
    eligible_products = get_eligible_products(customer)
    
    # Randomly assign 1 to 3 products
    owned_count = random.randint(1, 3)
    owned_products = random.sample(eligible_products, owned_count)
    
    for prod_id in owned_products:
        customer_products.append({
            "Customer_ID": cust_id,
            "Product_ID": prod_id,
            "Acquisition_Date": random_h1_2024_date().isoformat()
        })

# Create DataFrame
df_customer_products = pd.DataFrame(customer_products)

# Save to CSV
df_customer_products.to_csv("customer_products.csv", index=False)

print("✅ customer_products.csv created with Acquisition_Date in H1 2024.")



✅ customer_products.csv created with Acquisition_Date in H1 2024.


In [71]:
import pandas as pd
import sqlite3

# Load CSVs
df_customers = pd.read_csv("customers.csv")
df_transactions = pd.read_csv("transactions.csv")
df_products = pd.read_csv("products.csv")
df_feature_store = pd.read_csv("feature_store_enhanced.csv")  # From earlier step
df_customer_products = pd.read_csv("customer_products.csv")  # From earlier step

# Create SQLite DB
conn = sqlite3.connect("cross_selling.db")

# Write tables
df_customers.to_sql("customers", conn, if_exists="replace", index=False)
df_transactions.to_sql("transactions", conn, if_exists="replace", index=False)
df_products.to_sql("products", conn, if_exists="replace", index=False)
df_feature_store.to_sql("feature_store", conn, if_exists="replace", index=False)
df_customer_products.to_sql("customer_products", conn, if_exists="replace", index=False)

conn.close()
print("✅ Data loaded into cross_selling.db")


✅ Data loaded into cross_selling.db


## LLM

In [105]:
import sqlite3
import pandas as pd
from langchain.agents import tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI
from prompts import *
import re

conn = sqlite3.connect("cross_selling.db")

@tool
def fetch_customer_profile(name: str) -> str:
    """Fetch basic customer profile by full name."""
    df = pd.read_sql(f"SELECT * FROM customers WHERE First_Name || ' ' || Last_Name = '{name}'", conn)
    return df.to_json(orient="records") if not df.empty else "Customer not found."

@tool
def analyze_customer_behavior(customer_id: str) -> str:
    """Provides a detailed analysis of customer behavior, spending patterns, and financial signals."""
    df = pd.read_sql(f"SELECT * FROM feature_store WHERE Customer_ID = '{customer_id}'", conn)
    if df.empty:
        return "No behavior data found for this customer."
    
    row = df.iloc[0]
    insights = []

    # 1️⃣ Aggregation Period
    insights.append(f"Analysis based on {row['Aggregation_Days']} days of transaction data.")

    # 2️⃣ Spending Overview
    insights.append(f"Total spending during this period is ${row['Total_Spend']:.2f} across {row['Num_Transactions']} transactions.")
    insights.append(f"Average transaction amount is ${row['Avg_Txn_Amount']:.2f}, with a maximum single transaction of ${row['Max_Txn_Amount']:.2f}.")
    
    # 3️⃣ Key Spending Categories
    category_flags = []
    if row.get("Spend_Grocery", 0) > 500:
        category_flags.append(f"Grocery: ${row['Spend_Grocery']:.2f}")
    if row.get("Spend_Travel", 0) > 800:
        category_flags.append(f"Travel: ${row['Spend_Travel']:.2f}")
    if row.get("Spend_Fuel", 0) > 150:
        category_flags.append(f"Fuel: ${row['Spend_Fuel']:.2f}")
    if row.get("Spend_Medical", 0) > 200:
        category_flags.append(f"Medical: ${row['Spend_Medical']:.2f}")
    if row.get("Spend_Entertainment", 0) > 300:
        category_flags.append(f"Entertainment: ${row['Spend_Entertainment']:.2f}")

    if category_flags:
        insights.append("Significant spending detected in categories: " + "; ".join(category_flags))
    else:
        insights.append(f"Primary spending category is {row['Top_Spend_Category']}.")

    # 4️⃣ Income & Salary Patterns
    if row["Has_Salary_Credit"]:
        insights.append("Regular salary deposits detected, indicating stable income.")
    if row["Salary_to_Spend_Ratio"] > 0.5:
        insights.append(f"Healthy disposable income, with a Salary-to-Spend Ratio of {row['Salary_to_Spend_Ratio']:.2f}.")

    # 5️⃣ Financial Profile
    if row["Annual_Income"] > 100000:
        insights.append(f"High annual income: ${row['Annual_Income']}.")
    elif row["Annual_Income"] > 60000:
        insights.append(f"Moderate annual income: ${row['Annual_Income']}.")

    if row["Credit_Score"] >= 750:
        insights.append(f"Excellent credit score: {row['Credit_Score']}.")
    elif row["Credit_Score"] >= 700:
        insights.append(f"Good credit score: {row['Credit_Score']}.")
    else:
        insights.append(f"Credit score is {row['Credit_Score']}.")

    # 6️⃣ Spend Variability
    if row["Spend_Variability"] > 500:
        insights.append(f"High variability in spending, suggesting inconsistent transaction amounts.")
    else:
        insights.append(f"Consistent spending behavior with low variability.")

    # 7️⃣ Idle Balance Potential
    if row["Idle_Balance_Estimate"] > 5000:
        insights.append(f"Estimated idle balance of ${row['Idle_Balance_Estimate']:.2f}, indicating potential for savings or investment products.")

    return " | ".join(insights)

@tool
def fetch_product_catalog(dummy_input: str) -> str:
    """Returns the bank's product catalog for cross-selling."""
    df = pd.read_sql("SELECT * FROM products", conn)
    return df.to_json(orient="records")


@tool
def scientific_calculator(expression: str) -> str:
    """Performs safe scientific calculations. Provide expressions like '1250 / 28' or 'sqrt(256)'."""
    import math
    allowed_names = {k: v for k, v in math.__dict__.items() if not k.startswith("__")}
    allowed_names['abs'] = abs

    try:
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

@tool
def fetch_owned_products(customer_id: str) -> str:
    """
    Fetches the list of products currently owned by the customer.
    The LLM should avoid recommending these products again.
    """
    query = f"""
            SELECT p.Product_ID, p.Product_Name, p.Product_Type
            FROM customer_products cp
            JOIN products p ON cp.Product_ID = p.Product_ID
            WHERE cp.Customer_ID = '{customer_id}'
        """
    df = pd.read_sql(query, conn)
    
    if df.empty:
        return "This customer does not own any products currently."
    
    # Return a readable list
    owned_list = df[['Product_ID', 'Product_Name', 'Product_Type']].to_dict(orient='records')
    return f"Customer currently owns the following products: {owned_list}"

@tool
def fetch_schema_info(dummy_input: str = "") -> str:
    """
    Returns the pre‐built SCHEMA_INFO string describing your database schema.
    """
    return SCHEMA_INFO


@tool
def run_sql(sql_statements: str) -> str:
    """
    Executes one or more semicolon-separated SQL statements against `conn`.
    - SELECT queries: returns top 5 rows as a markdown table.
    - Other queries: returns success/failure status.
    """
    outputs = []
    for stmt in sql_statements.split(";"):
        stmt = stmt.strip()
        if not stmt:
            continue
        try:
            if stmt.lower().startswith("select"):
                df = pd.read_sql(stmt, conn)
                if df.empty:
                    outputs.append(f"✅ `{stmt}` returned no rows.")
                else:
                    table = df.to_markdown(index=False)
                    outputs.append(f"✅ Results for `{stmt}`:\n\n{table}")
            else:
                conn.execute(stmt)
                conn.commit()
                outputs.append(f"✅ Executed `{stmt}` successfully.")
        except Exception as e:
            outputs.append(f"❌ Error executing `{stmt}`:\n{str(e)}")
    return "\n\n".join(outputs)

@tool
def clean_sql_statement(raw_sql: str) -> str:
    """
    Strips out markdown fences (```sql``` or ```), any leading/trailing backticks,
    and excessive whitespace so you get a clean SQL string.
    """
    # Remove opening fences like ```sql or ``` and closing ```
    cleaned = re.sub(r"```(?:sql)?\s*", "", raw_sql)
    cleaned = cleaned.replace("```", "")
    # Collapse any multiple newlines/spaces
    cleaned = re.sub(r"\s*\n\s*", " ", cleaned)
    # Remove trailing semicolon if desired (we’ll handle semis in run_sql)
    cleaned = cleaned.strip().rstrip(";")
    return cleaned


In [106]:
fetch_customer_profile.description = "Fetch the customer's demographic and financial profile by full name."

fetch_product_catalog.description = "Retrieve the complete product catalog including features, target behaviors, eligibility criteria, and special offers."

fetch_owned_products.description = "Get a list of products already owned by the customer to avoid duplicate recommendations."

scientific_calculator.description = "Perform numeric calculations such as averages, ratios, or thresholds to support financial reasoning."

fetch_schema_info.description = (
    "Use this tool to retrieve the database schema (tables, columns, types) "
    "from the predefined SCHEMA_INFO string before using run_sql rool"
)

run_sql.description = (
    "Use this tool to execute raw SQL statements. "
    "Accepts one or multiple semicolon-separated queries. "
    "other statements return execution status."
    "use fetch_schema tool before using this tool"
)

analyze_customer_behavior.description = (
    "Use this ONLY to summarize customer behavior patterns for making product recommendations. "
    "Do NOT use this tool to answer specific data questions like amounts spent."
)

clean_sql_statement.description = (
    "Use this tool to take any raw SQL (even wrapped in markdown fences) "
    "and return a single clean SQL string, ready for execution." )

## Propensity-to-Buy Scorer

## Tool for propensity

### Eligibility criteria tool

In [93]:
@tool
def check_eligibility(customer_id: str, product_id: str) -> str:
    """
    Checks structured eligibility rules and returns 'eligible' or lists unmet criteria.
    """
    cust = pd.read_sql(f"SELECT Age, Annual_Income, Credit_Score FROM customers WHERE Customer_ID='{customer_id}'", conn).iloc[0]
    prod = pd.read_sql(f"SELECT min_age, min_income, min_credit_score FROM products WHERE Product_ID='{product_id}'", conn).iloc[0]
    fails = []
    if cust.Age < prod.min_age: fails.append(f"Age<{prod.min_age}")
    if cust.Annual_Income < prod.min_income: fails.append(f"Income<{prod.min_income}")
    if cust.Credit_Score < prod.min_credit_score: fails.append(f"CreditScore<{prod.min_credit_score}")
    owned = pd.read_sql(f"SELECT 1 FROM customer_products WHERE Customer_ID='{customer_id}' AND Product_ID='{product_id}'", conn)
    if not owned.empty: fails.append("AlreadyOwned")
    return "eligible" if not fails else "not eligible: " + ",".join(fails)

check_eligibility.description = (
    "Checks structured product eligibility rules (min_age,min_income,min_credit_score,not already owned)."
)


## Lifecycle Stage Detector

In [ ]:
@tool
def detect_stage(customer_id: str) -> str:
    row = pd.read_sql(f"SELECT Age, Customer_Since FROM customers WHERE Customer_ID='{customer_id}'", conn).iloc[0]
    tenure_years = (pd.Timestamp.now() - pd.to_datetime(row.Customer_Since)).days/365
    if row.Age < 25 and tenure_years<5: return "Student"
    if row.Age < 35: return "YoungProfessional"
    if row.Age < 50: return "Family"
    if row.Age < 60: return "PreRetiree"
    return "Retiree"

detect_stage.description = "Classifies customer life stage using age + tenure."


## Segment Enricher

In [ ]:
from sklearn.cluster import KMeans

features = pd.read_sql("SELECT * FROM feature_store_enhanced", conn).set_index('Customer_ID')
kmeans = KMeans(n_clusters=3, random_state=42).fit(features)
pd.Series(kmeans.labels_, index=features.index).to_csv("segments.csv")

@tool
def segment_customer(customer_id: str) -> str:
    seg = pd.read_csv("segments.csv", index_col=0).loc[customer_id]
    mapping = {0:"HighValue",1:"ValueShopper",2:"Occasional"}
    return mapping.get(seg, "Unknown")

segment_customer.description = "Assigns customer to one of three clusters trained on aggregated features."


## Bundle Optimizer

In [ ]:
import pulp

@tool
def optimize_bundle(customer_id: str, budget: float=0.0) -> str:
    # fetch eligible & scores
    prods = [pid for pid in pd.read_sql("SELECT Product_ID FROM products", conn)['Product_ID']
             if check_eligibility(customer_id,pid)=="eligible"]
    scores = {pid: float(score_propensity(customer_id,pid)) for pid in prods}
    # assume cost=1 per product for demo
    prob = pulp.LpProblem("bundle", pulp.LpMaximize)
    x = pulp.LpVariable.dicts('x', prods, cat='Binary')
    prob += pulp.lpSum([scores[p]*x[p] for p in prods])
    prob += pulp.lpSum([x[p] for p in prods]) <= 3  # max 3
    prob.solve()
    chosen = [p for p in prods if x[p].value()==1]
    return chosen

optimize_bundle.description = "Selects up to 3 eligible products maximizing propensity via 0/1 knapsack."


## Next-Best-Action Planner

In [ ]:
import json

@tool
def plan_next_actions(customer_id: str, _: str="") -> str:
    stage = detect_stage(customer_id)
    segment = segment_customer(customer_id)
    bundle = optimize_bundle(customer_id)
    return json.dumps({
        "stage": stage,
        "segment": segment,
        "recommendations": bundle
    })

plan_next_actions.description = (
    "Orchestrates detection, segmentation, and bundle optimization "
    "to output a next-best-action JSON."
)


In [ ]:
# OPENAI_DEPLOYMENT_ENDPOINT = "https://az-openai-document-question-answer-service.openai.azure.com/" 
# OPENAI_API_KEY = "5d24331966b648738e5003caad552df8" 
# OPENAI_API_VERSION = "2023-05-15"

# OPENAI_DEPLOYMENT_NAME = "az-gpt_35_model"
# OPENAI_MODEL_NAME="gpt-3.5-turbo"

# OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = "az-embedding_model" 
# OPENAI_ADA_EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

# encoding_name = "cl100k_base"

# llm = AzureChatOpenAI(
#                         temperature=0.1,
#                         deployment_name=OPENAI_DEPLOYMENT_NAME,
#                         model_name=OPENAI_MODEL_NAME,
#                         azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
#                         openai_api_version=OPENAI_API_VERSION,
#                         openai_api_key=OPENAI_API_KEY            
#                     )

In [118]:
OPENAI_DEPLOYMENT_ENDPOINT = "https://pkl-aa-dev-aiservices.openai.azure.com/" 
OPENAI_API_KEY = "AXEC3y1jC9ZNGCBB12NZwrpBSzScq1esexgvCXiqw7PaHE04vSMbJQQJ99BDACYeBjFXJ3w3AAABACOG4CMN" 
OPENAI_API_VERSION = "2024-12-01-preview"

OPENAI_DEPLOYMENT_NAME = "gpt-4o"
OPENAI_MODEL_NAME="gpt-4o"


llm = AzureChatOpenAI(
                        temperature=0.1,
                        deployment_name=OPENAI_DEPLOYMENT_NAME,
                        model_name=OPENAI_MODEL_NAME,
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY            
                    )

In [127]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor, create_react_agent
import textwrap

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", react_prompt),
        ("user", "{input}"),
        ("ai", "{agent_scratchpad}"),
    ]
)

tools = [fetch_customer_profile, analyze_customer_behavior, fetch_product_catalog, fetch_owned_products, fetch_schema_info, run_sql ,clean_sql_statement ]
# tools = [fetch_schema_info, run_sql  , clean_sql_statement]

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt   # Optional. If omitted, uses LangChain's default ReAct prompt
)

# 4. Agent Executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True
)

user_query = "How much did Kevin Fleming spend in total over how many days?"
# user_query = "Recommend products for Kevin Fleming based on his current products and expenditures."
# user_query = "Give me a view of all transactions by David Brown"
# user_query = "Give me customer info for DAvid Brown"
# user_query = "How many total customers bought more than 2 products ?"
response = agent_executor.invoke({"input": user_query})

output_text = response.get('output', '')

# Pretty print with wrapping at 100 characters





> Entering new AgentExecutor chain...
Thought: To determine how much Kevin Fleming spent in total and over how many days, I need to query the database. First, I need to fetch the schema information to understand the structure of the database.

Action: fetch_schema_info
Action Input: ''
You have access to the following database schema:

---

### 1. **customers**
Stores customer demographic and financial profile information.

| Column Name   | Type    | Description                                | Example         |
|---------------|---------|--------------------------------------------|-----------------|
| Customer_ID   | TEXT PK | Unique customer identifier                 | 'CUST0012'      |
| First_Name    | TEXT    | Customer's first name                      | 'David'         |
| Last_Name     | TEXT    | Customer's last name                       | 'Brown'         |
| Gender        | TEXT    | Customer's gender ('M' or 'F')             | 'M'             |
| Date_of_Birth | DATE  

In [125]:
def format_recommendation_summary(raw_text):
    # Remove initial label if present
    raw_text = raw_text.replace("**Recommendation Summary:**", "").strip()

    # Split by numbering (assuming '1.' and '2.' structure)
    parts = raw_text.split(" 2. ")
    rec_1 = parts[0].strip()
    rec_2 = parts[1].strip() if len(parts) > 1 else ""

    formatted_output = ""

    for idx, rec in enumerate([rec_1, rec_2], start=1):
        if not rec:
            continue
        # Split the recommendation into lines by bullet points
        lines = rec.split(" - ")
        title = lines[0].strip()
        bullets = lines[1:]

        formatted_output += f"{idx}. {title}\n"
        for bullet in bullets:
            bullet = bullet.replace("Eligibility Criteria", "Eligibility Criteria") \
                           .replace("Reason", "Reason") \
                           .replace("Benefit", "Benefit")
            formatted_output += f"   - {bullet.strip()}\n"
        formatted_output += "\n"

    return textwrap.dedent(formatted_output)

In [126]:
print(format_recommendation_summary(output_text))

1. Based on Kevin Fleming's financial profile, spending patterns, and existing products, I recommend the following products:

1. **Smart Shopper Card (P003)**:
   - **Reason**: Kevin has significant spending on fuel and groceries. This card offers 2% cashback on these categories, aligning well with his spending habits.
   - **Special Offer**: USD 50 cashback on the first USD 500 spend.

2. **CashBuilder Certificate (P016)**:
   - **Reason**: Kevin has an estimated idle balance of $93,333.20. This fixed deposit product offers a 4.75% fixed interest rate for 6 months, which can help him earn returns on his idle funds.
   - **Special Offer**: USD 25 bonus for auto-renewal.

3. **Lifestyle Bundle Plus (P017)**:
   - **Reason**: Kevin is an affluent customer with a high income and multiple existing products. This bundle combines credit, wealth, and travel perks, offering a premium experience.
   - **Special Offer**: USD 300 statement credit and concierge onboarding.

4. **HealthSecure Plan 